In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from peft import PeftModel
import torch
from unsloth import FastLanguageModel
import torch
import os
#os.environ["OPENAI_API_KEY"] = "sk-proj-DVG9lQbDED4pAGLLojr5T3BlbkFJAOrjyHbmt3WyfO2siJWW"
model_name = "gpt-3.5-turbo"
import openai
from datasets import load_dataset
from transformers import pipeline

# Set your OpenAI API key


# Load the Olympiadbench dataset
dataset = load_dataset("openai/Olympiadbench")

import openai
from datasets import load_dataset
from transformers import pipeline

# Set your OpenAI API key
openai.api_key = "your_api_key"

# Load the Olympiadbench dataset
# Load the saved dataset from mcts-r processed data
dataset = load_dataset("openai/Olympiadbench")


def evaluate(model, prompt, reference):
  # Use the OpenAI API to generate a response with GPT-3.5 Turbo
  response = openai.Completion.create(
      engine="text-davinci-003",  # Replace with "text-davinci-003" for GPT-3.5 Turbo
      prompt=prompt,
      max_tokens=100,  # Adjust for desired response length
      n=1,
      stop=None,
      temperature=0.7
  )
  generated_response = response.choices[0].text.strip()

  # Calculate metrics
  # 1. Accuracy (percentage of characters that match exactly)
  num_correct = sum(c1 == c2 for c1, c2 in zip(reference, generated_response))
  accuracy = num_correct / len(reference) * 100

  # 2. Precision (ratio of correctly predicted positive cases)
  # Identify true positives (correctly generated characters in the reference)
  true_positives = sum(1 for c in generated_response if c in reference)
  # Identify all predicted characters (all characters in generated_response)
  predicted_positives = len(generated_response)
  precision = (true_positives / predicted_positives) if predicted_positives > 0 else 0

  # 3. Recall (ratio of correctly identified relevant characters)
  # Identify true positives (correctly generated characters in the reference)
  true_positives = sum(1 for c in generated_response if c in reference)
  # Identify all relevant characters (all characters in the reference)
  relevant_characters = len(reference)
  recall = (true_positives / relevant_characters) if relevant_characters > 0 else 0

  # 4. F1 score (harmonic mean of precision and recall)
  f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

  return {
      "accuracy": accuracy,
      "precision": precision,
      "recall": recall,
      "f1": f1
  }


# Choose the model to evaluate
model_name = "text-davinci-003"  # Example: GPT-3.5 Turbo

# Create a pipeline for easier model interaction
model = pipeline("text-generation", model=model_name)

# Initialize empty dictionaries to store evaluation results
all_metrics = {"accuracy": [], "precision": [], "recall": [], "f1": []}

# Iterate through the dataset
for datapoint in dataset:
  prompt = datapoint["prompt"]
  reference = datapoint["reference"]
  # Evaluate on the current datapoint
  metrics = evaluate(model, prompt, reference)
  # Accumulate evaluation results
  for metric_name, value in metrics.items():
    all_metrics[metric_name].append(value)

# Calculate overall average scores for each metric
for metric_name, scores in all_metrics.items():
  average_score = sum(scores) / len(scores)
  print(f"Average {metric_name}: {average_score:.4f}")

# Calculate overall average scores for each metric
for metric_name, scores in all_metrics.items():
  average_score = sum(scores) / len(scores)
  print(f"Average {metric_name}: {average_score:.4f}")

In [ ]:
import openai
from datasets import load_dataset
import sympy  # Library for symbolic math operations

# Set your OpenAI API key
openai.api_key = "your_api_key"

# Load the Olympiadbench dataset
dataset = load_dataset("openai/Olympiadbench")


def evaluate(model, prompt, reference):
  # Use the OpenAI API to answer the math question
  response = openai.Completion.create(
      engine="text-davinci-003",  # Replace with "text-davinci-003" for GPT-3.5 Turbo
      prompt=prompt,
      max_tokens=100,  # Adjust for desired response length
      n=1,
      stop=None,
      temperature=0.7
  )
  generated_response = response.choices[0].text.strip()

  # Evaluate answer (considering symbolic representation)
  try:
    # Parse the reference answer (assuming it's a mathematical expression)
    reference_answer = sympy.sympify(reference)
    # Parse the generated response (assuming it's a mathematical expression)
    generated_answer = sympy.sympify(generated_response)
    # Check if the answers are symbolically equivalent
    answer_correct = sympy.simplify(reference_answer - generated_answer) == 0
  except (sympy.SympifyError, ValueError):
    # Handle cases where parsing fails (e.g., non-mathematical responses)
    answer_correct = False

  # Calculate accuracy (1 if answers are symbolically equivalent, 0 otherwise)
  accuracy = 1 if answer_correct else 0

  # Additional metrics (precision, recall, F1) might not be applicable here)
  # For these tasks, accuracy might be sufficient.

  return {"accuracy": accuracy}


# Choose the model to evaluate
model_name = "text-davinci-003"  # Example: GPT-3.5 Turbo

# Create a pipeline for easier model interaction (not strictly necessary here)
# model = pipeline("text-generation", model=model_name)

# Initialize empty dictionary to store evaluation results
all_metrics = {"accuracy": []}

# Iterate through the dataset
for datapoint in dataset:
  prompt = datapoint["prompt"]
  reference = datapoint["reference"]
  # Evaluate on the current datapoint
  metrics = evaluate(model, prompt, reference)
  # Accumulate evaluation results
  for metric_name, value in metrics.items():
    all_metrics[metric_name].append(value)

# Calculate overall average score for accuracy
average_accuracy = sum(all_metrics["accuracy"]) / len(all_metrics["accuracy"])
print(f"Average Accuracy: {average_accuracy:.4f}")